In [81]:
import pandas as pd
import numpy as np
import re
import xlsxwriter

In [82]:

def clean_estudiantes(data):
    data = data.iloc[7:]
    data = data.iloc[:, 1:]
    data = data.reset_index(drop=True)
    data.columns = data.iloc[0]
    data = data.iloc[1:]
    data = data.rename(columns=lambda x: re.sub(r'\W+', '_', x.strip().lower()))
    return data

In [83]:
CE1 = pd.read_excel(r'H:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Estudiantes\CE_1erSemestre.xlsx')

In [84]:
CE1 = clean_estudiantes(CE1)

In [85]:
grupo_evaluacion = ['proceso','semestre','sede','tipo_carrera','unidad_academica','codigo_carrera','nombre_carrera','codigo_actividad','nombre_actividad',
                         'paralelo','tipo_asig','rut_docente','nombre_docente','apaterno_docente','amaterno_docente','estado']
grupo_coberturas = ['proceso','semestre','sede','tipo_carrera','unidad_academica','codigo_carrera','nombre_carrera','codigo_actividad','nombre_actividad',
                         'paralelo','tipo_asig','rut_docente','nombre_docente','apaterno_docente','amaterno_docente']


# Comienza el DataFrame de Coberturas #

In [86]:
Cobertura_CE1 = CE1.groupby(grupo_evaluacion).size().reset_index(name='count')

In [87]:
Cobertura_CE1 = Cobertura_CE1.pivot_table(index=grupo_coberturas, columns='estado',values='count',fill_value=0).reset_index().rename(columns=lambda x: re.sub(r'\W+', '_', x.strip().lower()))

In [88]:
estados = ['finalizado','no_iniciada','pendiente']

In [89]:
Cobertura_CE1['total']= Cobertura_CE1[estados].sum(axis=1)

In [90]:
Cobertura_CE1['participacion'] = round((Cobertura_CE1['finalizado'] / Cobertura_CE1['total'])*100,2)

In [91]:
Cobertura_CE1

estado,proceso,semestre,sede,tipo_carrera,unidad_academica,codigo_carrera,nombre_carrera,codigo_actividad,nombre_actividad,paralelo,tipo_asig,rut_docente,nombre_docente,apaterno_docente,amaterno_docente,finalizado,no_iniciada,pendiente,total,participacion
0,2022,1,Antofagasta,Pregrado Trabajador,Facultad De Ciencias Empresariales,3710,Ingeniería De Ejecución En Control De Gestión,ICG71,Finanzas Ii,FA,Teoria,7616161-5,Ricardo Alex,Alvarez,Soto,4,18,0,22,18.18
1,2022,1,Antofagasta,Pregrado Trabajador,Facultad De Ciencias Empresariales,3710,Ingeniería De Ejecución En Control De Gestión,ICG72,Comercio Internacional,FA,Teoria,22255903-0,Luis Gerardo,García,Mendoza,15,14,0,29,51.72
2,2022,1,Antofagasta,Pregrado Trabajador,Facultad De Ciencias Empresariales,3710,Ingeniería De Ejecución En Control De Gestión,ICG73,S.I.A. I,FA,Teoria,9542203-9,Juan Pablo,Castro,Rodriguez,3,18,1,22,13.64
3,2022,1,Antofagasta,Pregrado Trabajador,Facultad De Ciencias Empresariales,3710,Ingeniería De Ejecución En Control De Gestión,ICG76,Técnicas De Auditoría Y De Control,FA,Teoria,8633756-8,Danilo Enrique,Segura,Riveros,0,19,0,19,0.00
4,2022,1,Antofagasta,Pregrado Trabajador,Facultad De Ciencias Empresariales,3710,Ingeniería De Ejecución En Control De Gestión,ICG77,Adiestramiento Computacional Profesional,FA,Teoria,9542203-9,Juan Pablo,Castro,Rodriguez,1,21,0,22,4.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,2022,1,Victoria,Pregrado Trabajador,Facultad De Ingeniería Y Arquitectura,64,Ingeniería De Ejecución En Prevención De Riesgos,IEP30,Higiene Industrial Iii,HHH,Teoria,8705874-3,Rufino Enrique,Avila,Salazar,6,51,2,59,10.17
499,2022,1,Victoria,Pregrado Trabajador,Facultad De Ingeniería Y Arquitectura,64,Ingeniería De Ejecución En Prevención De Riesgos,IEP37,Analisis De Normas Iso 9000-14000-18000,HHH,Teoria,14452655-4,Carlos Andrés,Crisóstomo,Martínez,6,52,1,59,10.17
500,2022,1,Victoria,Pregrado Trabajador,Facultad De Ingeniería Y Arquitectura,64,Ingeniería De Ejecución En Prevención De Riesgos,IEP38,Administ. Y Gestion De Rr.Hh.,HHH,Teoria,8261893-7,Ernesto,Salazar,Briones,17,38,3,58,29.31
501,2022,1,Victoria,Pregrado Trabajador,Facultad De Ingeniería Y Arquitectura,64,Ingeniería De Ejecución En Prevención De Riesgos,IEP39,Elaboracion Ejecucion De Proyectos,HHH,Teoria,11920036-9,Eduardo Marcelo,Gonzalez,Oyarzun,17,40,1,58,29.31


# Comienza la construcción del DataFrame de Resultados #

In [92]:
columnas_respuesta = [col for col in CE1.columns if col.startswith('respuesta')]
escala_puntuacion = {'Siempre': 3.0, 'Generalmente': 2.0, 'Pocas veces': 1.0, 'Nunca': 0.0, '--': 0.0}

In [93]:
CE1[columnas_respuesta] = CE1[columnas_respuesta].replace(escala_puntuacion)

In [94]:
columnas_excluidas = ['inicio_aplicacion', 'fin_aplicacion', 'estado', 'fecha_inicio', 'fecha_fin']

In [95]:
Resultados_CE1 = CE1.drop(columns=columnas_excluidas)

In [100]:
Resultados_CE1 = Resultados_CE1.groupby(grupo_coberturas).mean(columnas_respuesta).round(2).reset_index()

In [112]:
CE1_2022 = Cobertura_CE1.merge(
    right= Resultados_CE1,
    how = "outer",
    on = grupo_coberturas
)

In [113]:
respuestas_mediador = [col for col in CE1_2022.columns if col.startswith('respuesta_1_')]
respuestas_gestion = [col for col in CE1_2022.columns if col.startswith('respuesta_2_')]

In [116]:
CE1_2022['mediador']= CE1_2022[respuestas_mediador].mean(axis=1).round(2)

In [118]:
CE1_2022['gestion'] = CE1_2022[respuestas_gestion].mean(axis=1).round(2)

In [ ]:
CE1_2022

In [120]:
writer = pd.ExcelWriter('CE1_2022.xlsx', engine='xlsxwriter')
CE1_2022.to_excel(writer, sheet_name='CE12022', index=True)
writer.save()

C:\Users\aigna\AppData\Local\Temp\ipykernel_16440\149982443.py:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
